In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ast
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import helpers
import ollama

#### Get ESCO embeddings

In [ ]:
esco = pd.read_csv('datasets/skills_en.csv')

In [ ]:
esco_skills = esco['preferredLabel'].tolist()

In [ ]:
response = ollama.embed(model="nomic-embed-text", input=esco_skills)

In [ ]:
esco['embeddings'] = response['embeddings']

In [ ]:
esco.to_parquet('processed/esco_skills.parquet', index=False)

In [ ]:
esco = pd.read_parquet('processed/esco_skills.parquet')

#### Preprocess dice

In [3]:
dice = pd.read_csv('datasets/dice_com-job_us_sample.csv')

In [4]:
dice

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,SEE BELOW,418ff92580b270ef4e7c14f0ddfc36b4
1,https://www.dice.com/jobs/detail/Information-S...,University of Chicago/IT Services,Full Time,The University of Chicago has a rapidly growin...,Dice Id : 10114469,"Chicago, IL",Information Security Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"linux/unix, network monitoring, incident respo...",8aec88cba08d53da65ab99cf20f6f9d9
2,https://www.dice.com/jobs/detail/Business-Solu...,"Galaxy Systems, Inc.",Full Time,"GalaxE.SolutionsEvery day, our solutions affec...",Dice Id : CXGALXYS,"Schaumburg, IL",Business Solutions Architect,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Enterprise Solutions Architecture, business in...",46baa1f69ac07779274bcd90b85d9a72
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,Please see job description,3941b2f206ae0f900c4fba4ac0b18719
4,https://www.dice.com/jobs/detail/DevOps-Engine...,Matrix Resources,Full Time,Midtown based high tech firm has an immediate ...,Dice Id : matrixga,"Atlanta, GA",DevOps Engineer,48 minutes ago,Telecommuting not available|Travel not required,NaN,"Configuration Management, Developer, Linux, Ma...",45efa1f6bc65acc32bbbb953a1ed13b7
...,...,...,...,...,...,...,...,...,...,...,...,...
21995,https://www.dice.com/jobs/detail/Web-Designer-...,IAC Publishing,Full Time,Company Description We are searching for a ta...,Dice Id : 10112803,"Oakland, CA",Web Designer,3 weeks ago,Telecommuting not available|Travel not required,NaN,"UI/UX mobile apps, interaction design, digital...",86e27ce6b7e631e55d69d142c7d43df2
21996,https://www.dice.com/jobs/detail/Senior-Front-...,Omega Solutions Inc,Full Time,CONTACT - priya@omegasolutioninc.com / 408-45...,Dice Id : 10289500,"San Francisco, CA",Senior Front End Web Developer - Full Time at ...,3 weeks ago,Telecommuting not available|Travel not required,NaN,"JavaScript, HTML5, CSS3, Bootstrap, AJAX, Reac...",4287c7ee3317ccf1edd76e238cf8e584
21997,https://www.dice.com/jobs/detail/QA-Analyst-Sa...,San Francisco Health Plan,Full Time,Do you take pride in your work knowing that th...,Dice Id : 10115761,"San Francisco, CA",QA Analyst,2 weeks ago,Telecommuting not available|Travel not required,NaN,"SDLC, ALM, SQL, T-SQL, RedGate, Team Foundatio...",d7512f0181d69f83f96db38cd77a4d08
21998,https://www.dice.com/jobs/detail/Tech-Lead%252...,IAC Publishing,Full Time,Company Description What We Can Offer YouAs th...,Dice Id : 10112803,"Oakland, CA",Tech Lead-Full Stack,2 weeks ago,Telecommuting not available|Travel not required,NaN,"Python, Ruby, Go, Clojure, Java, NoSQL-Databas...",ec375268b494b3bcbed1635d64226112


In [5]:
dice['skills'] = dice['skills'].fillna('')

In [6]:
dice['skills'] = dice['skills'].str.lower().str.strip()

In [7]:
dice[dice['skills'].str.contains('see')]

,advertiserurl,company,employmenttype_jobstatus,jobdescription,jobid,joblocation_address,jobtitle,postdate,shift,site_name,skills,uniq_id
0,https://www.dice.com/jobs/detail/AUTOMATION-TE...,"Digital Intelligence Systems, LLC","C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...",Looking for Selenium engineers...must have sol...,Dice Id : 10110693,"Atlanta, GA",AUTOMATION TEST ENGINEER,1 hour ago,Telecommuting not available|Travel not required,NaN,see below,418ff92580b270ef4e7c14f0ddfc36b4
3,https://www.dice.com/jobs/detail/Java-Develope...,TransTech LLC,Full Time,Java DeveloperFull-time/direct-hireBolingbrook...,Dice Id : 10113627,"Bolingbrook, IL","Java Developer (mid level)- FT- GREAT culture,...",2 weeks ago,Telecommuting not available|Travel not required,NaN,please see job description,3941b2f206ae0f900c4fba4ac0b18719
9,https://www.dice.com/jobs/detail/Application-S...,"VanderHouwen & Associates, Inc.","C2H W2, Contract to hire",SummaryOur client is the leading provider of o...,Dice Id : vhassoc,"Seattle, WA",Application Support Engineer,7 hours ago,Telecommuting not available|Travel not required,NaN,(see job description),95c9127e2770172f454f3b83981eaa88
15,https://www.dice.com/jobs/detail/Windows-Syste...,Eastridge Workforce Solutions,"Full Time, contract-to-hire",Windows Systems Administrator The Windows Syst...,Dice Id : eastsd,"Las Vegas, NV",Windows System Administrator,7 hours ago,Telecommuting not available|Travel not required,NaN,(see job description),9e5704d08bc07ddb6df9ef98b223b036
19,https://www.dice.com/jobs/detail/Java-Architec...,Fahrenheit IT Staffing & Consulting,"Full Time, Direct Hire",Our client in the healthcare software industry...,Dice Id : 10111360,"Denver, CO","Java Architect - Denver, CO - Fulltime",7 hours ago,Telecommuting not available|Travel not required,NaN,(see job description),e4f57bc5366124a0a47cac27f557f9ec
...,...,...,...,...,...,...,...,...,...,...,...,...
20764,https://www.dice.com/jobs/detail/Mobile%2526%2...,Wipro Ltd.,Full Time,JOB DESCRIPTION:8 to 12 years of experience in...,Dice Id : 10110759,"Milpitas, CA",Mobile-Automation Architect,1 week ago,Telecommuting not available|Travel not required,NaN,"automation-framework, java-open source, mobile...",0837cbd9d1811291aca349ab6b04515d
21005,https://www.dice.com/jobs/detail/Quality-and-R...,Silver Spring Networks,Full Time,We are a growing company that has gained soli...,Dice Id : RTX14ae36,"San Jose, CA",Quality and Reliability Engineer,1 week ago,Telecommuting not available|Travel not required,NaN,"bsee, quality test engineer, rf, development, ...",1752d6703ad529423b6b41d3c367cc61
21476,https://www.dice.com/jobs/detail/Mobile-Automa...,Wipro Ltd.,"Full Time, Permanent",Wipro Ltd (NYSE:WIT) is a global information t...,Dice Id : 10110759,"Milpitas, CA",Mobile Automation Test Lead/Engineer-CA/WI,1 week ago,Telecommuting not available|Travel not required,NaN,"mobile application testing, qa scripts, test c...",4e35afef7d1faf05d971ca4e72f25516
21568,https://www.dice.com/jobs/detail/Applications-...,Macy's,Full Time,"Job Overview:In this role, the Applications Ar...",Dice Id : 10290276a,"San Francisco, CA","Applications Architect, Selection (macys.com)",3 weeks ago,Telecommuting not available|Travel not required,NaN,see job overview.,4fa8a123b307765c21beef51693df0b5


In [8]:
dice[~dice['skills'].str.contains('see')]['skills']

1        linux/unix, network monitoring, incident respo...
2        enterprise solutions architecture, business in...
4        configuration management, developer, linux, ma...
5                     fico, ar, ap, asset management, haha
6        cisco, dns, http, networking, network engineer...
                               ...                        
21995    ui/ux mobile apps, interaction design, digital...
21996    javascript, html5, css3, bootstrap, ajax, reac...
21997    sdlc, alm, sql, t-sql, redgate, team foundatio...
21998    python, ruby, go, clojure, java, nosql-databas...
21999                                                 null
Name: skills, Length: 21787, dtype: object

In [ ]:
dice_w_skills = helpers.get_list(data=dice, 
                                 col='jobdescription', 
                                 doc_type='job', 
                                 max_workers=10,
                                 model="mistral:instruct")

  3%|▎         | 737/22000 [15:17<7:40:56,  1.30s/it] 

Error on row 736: malformed node or string on line 1: <ast.List object at 0x00000224203C7490>


  4%|▍         | 828/22000 [17:27<20:03:22,  3.41s/it]

Error on row 827: unmatched ']' (<unknown>, line 3)


  4%|▍         | 896/22000 [19:01<6:32:45,  1.12s/it] 

...

 37%|███▋      | 8189/22000 [3:02:16<3:47:32,  1.01it/s]

Error on row 8188: malformed node or string on line 1: <ast.List object at 0x0000022421B4D720>


 40%|████      | 8895/22000 [3:17:34<3:12:18,  1.14it/s]

Error on row 8894: unterminated string literal (detected at line 1) (<unknown>, line 1)


 42%|████▏     | 9177/22000 [3:23:16<5:34:33,  1.57s/it]

In [ ]:
dice.to_parquet('processed/dice_job_descriptions_with_skills.parquet', index=False)

In [ ]:
dice_w_skills

In [9]:
job = dice['jobdescription'][0]

In [10]:
prompt = helpers.get_prompt(job, 'job description')

In [11]:
prompt

'Invalid type'

#### Preprocessing Resume

In [ ]:
resume = pd.read_csv('datasets/Resume.csv')

In [ ]:
job = dice['jobdescription'][0]

In [ ]:
prompt = helpers.get_prompt(job, 'job description')

In [ ]:
print(prompt)

In [ ]:
resp = helpers.get_response(prompt, model="mistral:instruct")

In [ ]:
ast.literal_eval(resp['response'].strip())

In [ ]:
resume_w_skills = helpers.get_list(data=resume, 
                                   col='Resume_str',
                                   doc_type='resume', 
                                   max_workers=10,
                                   model="mistral:instruct")

In [ ]:
resume_w_skills.to_parquet('processed/resume_w_skills.parquet', index=False)

#### debugging

In [ ]:
resume = pd.read_parquet('processed/resume_w_skills.parquet')

In [ ]:
resume = helpers.fill_missing_skills(resume, esco)

In [ ]:
resume

In [ ]:
resume['length'] = resume['Resume_str'].str.len()

In [ ]:
resume[['Resume_str', 'extracted_skills', 'length']]

In [ ]:
print(resume['Resume_str'].iloc[94])

In [ ]:
prompt = helpers.get_prompt(resume['Resume_str'].iloc[45], 'resume')

In [ ]:
response = helpers.get_response(prompt)

In [ ]:
response['response'].strip()

In [ ]:
skills = resume['extracted_skills'].iloc[0]

In [ ]:
embeddings = esco['embeddings']

In [ ]:
embeddings = embeddings.tolist()

In [ ]:
embedding = helpers.get_embedding(skills[0])

In [ ]:
cosine_similarity([embedding], embeddings)

In [ ]:
matched = helpers.match_closest_skills(skills, esco, threhold=0.8)

In [ ]:
matched_list, matched_scores = matched

In [ ]:
matched_list

In [ ]:
matched_scores

In [ ]:
comp_df = pd.DataFrame(columns=['skill', 'matched_skill', 'score'], data=list(zip(skills, matched_list, matched_scores)))

In [ ]:
comp_df

In [ ]:
resume_matched = helpers.match_all_skills_con(resume, esco, threshold=0.8)

In [ ]:
resume_matched

In [ ]:
resume_matched.to_parquet('processed/resume_matched.parquet', index=False)